# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 18 2023 1:35PM,261344,19,MSP220884,"Methapharm, Inc.",Released
1,Apr 18 2023 1:25PM,261343,19,23-T0190,"NAPP Technologies, LLC",Released
2,Apr 18 2023 1:20PM,261342,10,MSP220879,"Methapharm, Inc.",Released
3,Apr 18 2023 1:20PM,261342,10,MSP220880,"Methapharm, Inc.",Released
4,Apr 18 2023 1:20PM,261342,10,MSP220881,"Methapharm, Inc.",Released
5,Apr 18 2023 1:20PM,261342,10,MSP220878,"Methapharm, Inc.",Released
6,Apr 18 2023 1:20PM,261342,10,MSP220882,"Methapharm, Inc.",Released
7,Apr 18 2023 1:19PM,261341,18,WHC-666227/664630,WHC Labs bvba,Released
8,Apr 18 2023 1:11PM,261340,16,9392170,Sartorius Bioprocess Solutions,Released
9,Apr 18 2023 1:08PM,261339,15,30015826,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,261340,Released,1
40,261341,Released,1
41,261342,Released,5
42,261343,Released,1
43,261344,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261340,NaN,NaN,1.0
261341,NaN,NaN,1.0
261342,NaN,NaN,5.0
261343,NaN,NaN,1.0
261344,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16.0,26.0,11.0
261225,27.0,0.0,3.0
261229,0.0,0.0,34.0
261234,0.0,0.0,1.0
261238,5.0,13.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
260872,16,26,11
261225,27,0,3
261229,0,0,34
261234,0,0,1
261238,5,13,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,26,11
1,261225,27,0,3
2,261229,0,0,34
3,261234,0,0,1
4,261238,5,13,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,260872,16,26,11
1,261225,27,,3
2,261229,,,34
3,261234,,,1
4,261238,5,13,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc."
1,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC"
2,Apr 18 2023 1:20PM,261342,10,"Methapharm, Inc."
7,Apr 18 2023 1:19PM,261341,18,WHC Labs bvba
8,Apr 18 2023 1:11PM,261340,16,Sartorius Bioprocess Solutions
9,Apr 18 2023 1:08PM,261339,15,"Alliance Pharma, Inc."
11,Apr 18 2023 1:08PM,261328,15,"Brookfield Pharmaceuticals, LLC"
12,Apr 18 2023 1:07PM,261326,15,"Person & Covey, Inc."
13,Apr 18 2023 1:07PM,261323,15,"Alliance Pharma, Inc."
14,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",,,1
1,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",,,1
2,Apr 18 2023 1:20PM,261342,10,"Methapharm, Inc.",,,5
3,Apr 18 2023 1:19PM,261341,18,WHC Labs bvba,,,1
4,Apr 18 2023 1:11PM,261340,16,Sartorius Bioprocess Solutions,,,1
5,Apr 18 2023 1:08PM,261339,15,"Alliance Pharma, Inc.",,,2
6,Apr 18 2023 1:08PM,261328,15,"Brookfield Pharmaceuticals, LLC",,,1
7,Apr 18 2023 1:07PM,261326,15,"Person & Covey, Inc.",,,1
8,Apr 18 2023 1:07PM,261323,15,"Alliance Pharma, Inc.",,,1
9,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",,,34


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1,,
1,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",1,,
2,Apr 18 2023 1:20PM,261342,10,"Methapharm, Inc.",5,,
3,Apr 18 2023 1:19PM,261341,18,WHC Labs bvba,1,,
4,Apr 18 2023 1:11PM,261340,16,Sartorius Bioprocess Solutions,1,,
5,Apr 18 2023 1:08PM,261339,15,"Alliance Pharma, Inc.",2,,
6,Apr 18 2023 1:08PM,261328,15,"Brookfield Pharmaceuticals, LLC",1,,
7,Apr 18 2023 1:07PM,261326,15,"Person & Covey, Inc.",1,,
8,Apr 18 2023 1:07PM,261323,15,"Alliance Pharma, Inc.",1,,
9,Apr 18 2023 1:01PM,261229,15,"Alliance Pharma, Inc.",34,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1,,
1,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",1,,
2,Apr 18 2023 1:20PM,261342,10,"Methapharm, Inc.",5,,
3,Apr 18 2023 1:19PM,261341,18,WHC Labs bvba,1,,
4,Apr 18 2023 1:11PM,261340,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1.0,NaN,NaN
1,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",1.0,NaN,NaN
2,Apr 18 2023 1:20PM,261342,10,"Methapharm, Inc.",5.0,NaN,NaN
3,Apr 18 2023 1:19PM,261341,18,WHC Labs bvba,1.0,NaN,NaN
4,Apr 18 2023 1:11PM,261340,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 18 2023 1:35PM,261344,19,"Methapharm, Inc.",1.0,0.0,0.0
1,Apr 18 2023 1:25PM,261343,19,"NAPP Technologies, LLC",1.0,0.0,0.0
2,Apr 18 2023 1:20PM,261342,10,"Methapharm, Inc.",5.0,0.0,0.0
3,Apr 18 2023 1:19PM,261341,18,WHC Labs bvba,1.0,0.0,0.0
4,Apr 18 2023 1:11PM,261340,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3919771,50.0,5.0,1.0
15,1567770,42.0,0.0,27.0
16,522619,1.0,1.0,0.0
18,261341,1.0,0.0,0.0
19,2351673,34.0,26.0,5.0
20,522172,12.0,26.0,16.0
21,261322,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3919771,50.0,5.0,1.0
1,15,1567770,42.0,0.0,27.0
2,16,522619,1.0,1.0,0.0
3,18,261341,1.0,0.0,0.0
4,19,2351673,34.0,26.0,5.0
5,20,522172,12.0,26.0,16.0
6,21,261322,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,50.0,5.0,1.0
1,15,42.0,0.0,27.0
2,16,1.0,1.0,0.0
3,18,1.0,0.0,0.0
4,19,34.0,26.0,5.0
5,20,12.0,26.0,16.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,50.0
1,15,Released,42.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,34.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,1.0,27.0,0.0,0.0,5.0,16.0,0.0
Executing,5.0,0.0,1.0,0.0,26.0,26.0,1.0
Released,50.0,42.0,1.0,1.0,34.0,12.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,1.0,27.0,0.0,0.0,5.0,16.0,0.0
1,Executing,5.0,0.0,1.0,0.0,26.0,26.0,1.0
2,Released,50.0,42.0,1.0,1.0,34.0,12.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,1.0,27.0,0.0,0.0,5.0,16.0,0.0
1,Executing,5.0,0.0,1.0,0.0,26.0,26.0,1.0
2,Released,50.0,42.0,1.0,1.0,34.0,12.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()